In [2]:
import pandas as pd
import numpy as np

import random

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import joblib

from sklearn.inspection import permutation_importance
import shap
import random

/Users/birdslab/Desktop/Krithikaa/fluxes/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
random.seed(10)

Function to load the datasets, preprocess and format the data


Input: None


Output: Return the preprocessed dataframes

In [4]:
def load_dataset():

    #load T and P
    t = pd.read_csv('./transcriptome/compare_transcriptome.csv')
    p = pd.read_csv('./proteome/compare_proteome.csv')

    #remove the NaN values from P
    p.dropna(inplace=True)

    #set the knockout name as the index
    t.index = t['Unnamed: 0']
    p.index = p['Unnamed: 0']

    #remove the knockout name column from the dataframe
    t = t.iloc[:, 1:]
    p = p.iloc[:, 2:]

    #load the fluxomics datasets and set the index same as T or P
    fluxes_p = pd.read_csv('./fluxes/Fluxes_glpk_seed1.csv', header=None)
    fluxes_p = fluxes_p.T
    fluxes_p.index = p.index 

    fluxes_t = pd.read_csv('./fluxes/Fluxes_t_glpk.csv', header=None)
    fluxes_t  = fluxes_t.T
    fluxes_t.index = t.index

    #min-max normalization of fluxomics dataframe   
    fluxes_p = (fluxes_p - fluxes_p.min()) / (fluxes_p.max() - fluxes_p.min())

    fluxes_t = (fluxes_t - fluxes_t.min()) / (fluxes_t.max() - fluxes_t.min())

    #remove the NaN columns from the normalized fluxomics dataframe
    fluxes_p.dropna(axis=1, inplace=True)
    fluxes_t.dropna(axis=1, inplace=True)

    #min-max normalization of T and P dataframe
    p = (p - p.min())/(p.max() - p.min())
    t = (t - t.min())/(t.max() - t.min())

    #load the growth rates and set the index same as T or P
    gr_measured_ML = pd.read_csv('./fluxes/compare_transcriptome_measured_gr.csv')
    gr_measured_ML.index = t.index

    y = pd.read_csv('./fluxes/compare_measured_gr.csv')
    y.index = y['ORF PROT']

    return [t, fluxes_t, p, fluxes_p, gr_measured_ML, y]

Function to filter knockouts as low or high growth based on a given percentile


Input: Liquid GR, Solid GR, Percentile for splitting


Output: Modified GR dataframe with a class column [0 as low and 2 as high]

In [5]:
def filter_ko(gr_measured_ML, y, a_per, b_per):
    
    #find the percentile for liquid GR
    a = np.percentile(gr_measured_ML['0'], a_per)
    b = np.percentile(gr_measured_ML['0'], b_per)

    y_t = []
    for i in gr_measured_ML['0']:
        if i < a:
            y_t.append(0)
        elif i>a and i<b:
            y_t.append(1)
        else:
            y_t.append(2)

    #append class column based on the calculated percentile
    gr_measured_ML['class'] = y_t


    #find the percentile for solid GR
    a = np.percentile(y['SM'], a_per)
    b = np.percentile(y['SM'], b_per)

    y_p = []
    for i in y['SM']:
        if i < a:
            y_p.append(0)
        elif i>a and i<b:
            y_p.append(1)
        else:
            y_p.append(2)

    #append class column based on the calculated percentile
    y['class'] = y_p

    #filter medium gr KOs
    gr_measured_ML = gr_measured_ML[gr_measured_ML['class'] != 1]
    y = y[y['class'] != 1]

    return [gr_measured_ML, y]

Function to combine the transcriptomics, proteomics and fluxomics dataset into a single dataframe


Input: Transcriptomics, Proteomics, Fluxomics derived from transcriptomics, Fluxomics derived from proteomics


Output: Dataframe 

In [6]:
def generate_combined_dataset(t, p, fluxes_t, fluxes_p):

    X = pd.concat([t, fluxes_t, p, fluxes_p], axis=1)
    X.columns = X.columns.astype(str)

    return X

Function to split the dataset into train, test and validation 

Input: y dataframes

Output: Train, test and validation dataframes

In [7]:
def split_data(y):

    # est: 30% of the total data
    test_size = int(0.3*y.shape[0])

    #train: 80% of the remaining data
    train_size = int(0.8*(y.shape[0] - test_size))

    valid_size = int(0.2*(y.shape[0] - test_size))

    #randomly sample train_size number of rows, ensuring equal split of each class
    y_train = y.groupby('class', group_keys=False).apply(lambda x: x.sample(n=train_size//2, random_state=42))

    #filter the remaining indices
    train_valid = y.loc[list(set(y.index).difference(set(y_train.index))), :]

    #randomly sample valid_size number of rows from the remaining rows
    y_valid = train_valid.groupby('class', group_keys=False).apply(lambda x: x.sample(n=valid_size//2, random_state=42))

    y_test = y.loc[list(set(y.index).difference(set(y_valid.index).union(set(y_train.index)))), :]

    return [y_train, y_test, y_valid]

Function to extract features from the models

Input: model, dataframe with the gene/protein names, x validation dataframe, file name

In [8]:
def feature_importance(model, labels, X_valid, file_name, svm=False):

    if not svm:
        explainer = shap.Explainer(model.best_estimator_)
        shap_values = explainer.shap_values(X_valid)
        shap_summary = [np.mean(i) for i in np.abs(shap_values[0])]
        shap_summary_pd = pd.DataFrame({'Feature': X_valid.columns, 'SHAP values': shap_summary})
        shap_summary_pd = shap_summary_pd.sort_values('SHAP values', ascending=False)
        shap_summary_pd['Feature'] = [labels[labels['Gene Names (ordered locus)'] == i]['Entry'].values[0] for i in shap_summary_pd['Feature']]
        shap_summary_pd.to_csv("./pathway analysis/" + file_name + ".csv")
    else:

        #initialize the shap explainer
        explainer = shap.Explainer(model.best_estimator_.predict, X_valid)

        #calculate the shap values
        shap_values = explainer.shap_values(X_valid)
        shap_summary = [np.mean(i) for i in np.abs(shap_values[0])]

        #convert the values to a dataframe
        shap_summary_pd = pd.DataFrame({'Feature': X_valid.columns, 'SHAP values': shap_summary})

        #sort in ascending order
        shap_summary_pd = shap_summary_pd.sort_values('SHAP values', ascending=False)

        #set gene name for the features
        shap_summary_pd['Feature'] = [labels[labels['Gene Names (ordered locus)'] == i]['Entry'].values[0] for i in shap_summary_pd['Feature']]
        
        return shap_summary_pd
        # #save the features as a csv
        # shap_summary_pd.to_csv("./pathway analysis/" + file_name + ".csv")

In [9]:
labels = pd.read_excel("./gene-protein-yeast.xlsx")
double_KO = pd.read_csv("../Krithikaa/yeast_datasets/yeast_gstf_dataset.csv")

/Users/birdslab/Desktop/Krithikaa/fluxes/.venv/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
#load and preprocess data
[t, fluxes_t, p, fluxes_p, gr_measured_ML, y] = load_dataset()

percent = 30

#filter based on percentile
[gr_measured_ML, y] = filter_ko(gr_measured_ML, y, percent, 100 - percent)

#concatenate the dataframes
X = generate_combined_dataset(t, p, fluxes_t, fluxes_p)

#split transcriptomics dataset into train, test and validation
[y_train_liq, y_test_liq, y_valid_liq] = split_data(gr_measured_ML)
X_train_liq = X.loc[y_train_liq.index, :]
X_test_liq = X.loc[y_test_liq.index, :]
X_valid_liq = X.loc[y_valid_liq.index, :]

y_valid_liq = y_valid_liq['class']
y_train_liq = y_train_liq['class']
y_test_liq = y_test_liq['class']

#split proteomics dataset into train, test and validation
[y_train_solid, y_test_solid, y_valid_solid] = split_data(y)
X_train_solid = X.loc[y_train_solid.index, :]
X_test_solid = X.loc[y_test_solid.index, :]
X_valid_solid = X.loc[y_valid_solid.index, :]

y_train_solid = y_train_solid['class']
y_test_solid = y_test_solid['class']
y_valid_solid = y_valid_solid['class']

#subset train and validation for transcriptomics
x_train = X_train_liq.iloc[:, 0:1826].loc[:, double_KO.columns[1:]]
x_valid = X_valid_liq.iloc[:, 0:1826].loc[:, double_KO.columns[1:]]

svm = joblib.load("svm_t_glpk_"+str(percent)+".joblib")
result = permutation_importance(svm, x_valid, y_valid_liq, n_repeats=10, random_state=0, n_jobs=-1)
perm_imp_t_liq = pd.DataFrame({'Feature': x_valid.columns, 'Permutation Importance': result.importances_mean}).sort_values('Permutation Importance', ascending=False)

/var/folders/1g/phgxzfj511q1zyh71ggb7hxw0000gq/T/ipykernel_27149/3380375233.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y_train = y.groupby('class', group_keys=False).apply(lambda x: x.sample(n=train_size//2, random_state=42))
/var/folders/1g/phgxzfj511q1zyh71ggb7hxw0000gq/T/ipykernel_27149/3380375233.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y_valid = train_valid.groupby('class', group_

In [17]:
perm_imp_t_liq.to_dict('records')

[{'Feature': 'YBR029C', 'Permutation Importance': 0.004878048780487809},
 {'Feature': 'YPR019W', 'Permutation Importance': 0.003658536585365857},
 {'Feature': 'YMR091C', 'Permutation Importance': 0.002439024390243916},
 {'Feature': 'YDR292C', 'Permutation Importance': 0.002439024390243916},
 {'Feature': 'YGL255W', 'Permutation Importance': 0.0024390243902439046},
 {'Feature': 'YBL039C', 'Permutation Importance': 3.3306690738754695e-17},
 {'Feature': 'YMR289W', 'Permutation Importance': 2.2204460492503132e-17},
 {'Feature': 'YHR205W', 'Permutation Importance': 2.2204460492503132e-17},
 {'Feature': 'YLR209C', 'Permutation Importance': 1.1102230246251566e-17},
 {'Feature': 'YML110C', 'Permutation Importance': 1.1102230246251566e-17},
 {'Feature': 'YBR006W', 'Permutation Importance': 1.1102230246251566e-17},
 {'Feature': 'YKL088W', 'Permutation Importance': 0.0},
 {'Feature': 'YHR037W', 'Permutation Importance': 0.0},
 {'Feature': 'YOR007C', 'Permutation Importance': 0.0},
 {'Feature': 'YJ

In [38]:
coef = pd.DataFrame()
splits = [10, 20, 30, 40, 50]

for s in splits:
        
        #load and preprocess data
        [t, fluxes_t, p, fluxes_p, gr_measured_ML, y] = load_dataset()

        percent = s

        #filter based on percentile
        [gr_measured_ML, y] = filter_ko(gr_measured_ML, y, percent, 100 - percent)

        #concatenate the dataframes
        X = generate_combined_dataset(t, p, fluxes_t, fluxes_p)

        #split transcriptomics dataset into train, test and validation
        [y_train_liq, y_test_liq, y_valid_liq] = split_data(gr_measured_ML)
        X_train_liq = X.loc[y_train_liq.index, :]
        X_test_liq = X.loc[y_test_liq.index, :]
        X_valid_liq = X.loc[y_valid_liq.index, :]

        y_valid_liq = y_valid_liq['class']
        y_train_liq = y_train_liq['class']
        y_test_liq = y_test_liq['class']

        #split proteomics dataset into train, test and validation
        [y_train_solid, y_test_solid, y_valid_solid] = split_data(y)
        X_train_solid = X.loc[y_train_solid.index, :]
        X_test_solid = X.loc[y_test_solid.index, :]
        X_valid_solid = X.loc[y_valid_solid.index, :]

        y_train_solid = y_train_solid['class']
        y_test_solid = y_test_solid['class']
        y_valid_solid = y_valid_solid['class']

        print(s)

        #subset train and validation for transcriptomics
        x_train = X_train_liq.iloc[:, 0:1826].loc[:, double_KO.columns[1:]]
        x_valid = X_valid_liq.iloc[:, 0:1826].loc[:, double_KO.columns[1:]]

        loaded_model = joblib.load("svm_t_glpk_"+str(percent)+".joblib")
        result = permutation_importance(loaded_model, x_valid, y_valid_liq, n_repeats=10, random_state=0, n_jobs=-1)
        features = {loaded_model.best_estimator_.feature_names_in_[i]: result.importances_mean[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)

        loaded_model = joblib.load("knn_t_glpk_"+str(percent)+".joblib")
        result = permutation_importance(loaded_model, x_valid, y_valid_liq, n_repeats=10, random_state=0, n_jobs=-1)
        features = {loaded_model.best_estimator_.feature_names_in_[i]: result.importances_mean[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)

        loaded_model = joblib.load("lasso_log_t_glpk_"+str(percent)+".joblib")
        features = {loaded_model.best_estimator_.feature_names_in_[i]: loaded_model.best_estimator_.coef_[0][i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)

        loaded_model = joblib.load("elasticnet_log_t_glpk_"+str(percent)+".joblib")
        features = {loaded_model.best_estimator_.feature_names_in_[i]: loaded_model.best_estimator_.coef_[0][i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)


        transcriptome = X_train_liq.iloc[:, 0:1826].loc[:, double_KO.columns[1:]]
        flux = X_train_liq.iloc[:, 1826: 3776]
        x_train = pd.concat([transcriptome, flux], axis=1)
        x_valid = X_valid_liq.iloc[:, 0:3776].loc[:, x_train.columns]

        loaded_model = joblib.load("svm_t_f_glpk_"+str(percent)+".joblib")
        result = permutation_importance(loaded_model, x_valid, y_valid_liq, n_repeats=10, random_state=0, n_jobs=-1)
        features = {loaded_model.best_estimator_.feature_names_in_[i]: result.importances_mean[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)

        loaded_model = joblib.load("knn_t_f_glpk_"+str(percent)+".joblib")
        result = permutation_importance(loaded_model, x_valid, y_valid_liq, n_repeats=10, random_state=0, n_jobs=-1)
        features = {loaded_model.best_estimator_.feature_names_in_[i]: result.importances_mean[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)

        loaded_model = joblib.load("lasso_log_t_f_glpk_"+str(percent)+".joblib")
        features = {loaded_model.best_estimator_.feature_names_in_[i]: loaded_model.best_estimator_.coef_[0][i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)

        loaded_model = joblib.load("elasticnet_log_t_f_glpk_"+str(percent)+".joblib")
        features = {loaded_model.best_estimator_.feature_names_in_[i]: loaded_model.best_estimator_.coef_[0][i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)


        x_train = X_train_solid.iloc[:, 3776:5602].loc[:, double_KO.columns[1:]]
        x_valid = X_valid_solid.iloc[:, 3776:5602].loc[:, double_KO.columns[1:]]
        x_train.dropna(inplace=True)
        x_valid.dropna(inplace=True)

        loaded_model = joblib.load("svm_p_glpk_"+str(percent)+".joblib")
        result = permutation_importance(loaded_model, x_valid, y_valid_solid.loc[x_valid.index], n_repeats=10, random_state=0, n_jobs=-1)
        features = {loaded_model.best_estimator_.feature_names_in_[i]: result.importances_mean[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)

        loaded_model = joblib.load("knn_p_glpk_"+str(percent)+".joblib")
        result = permutation_importance(loaded_model, x_valid, y_valid_solid.loc[x_valid.index], n_repeats=10, random_state=0, n_jobs=-1)
        features = {loaded_model.best_estimator_.feature_names_in_[i]: result.importances_mean[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)

        loaded_model = joblib.load("lasso_log_p_glpk_"+str(percent)+".joblib")
        features = {loaded_model.best_estimator_.feature_names_in_[i]: loaded_model.best_estimator_.coef_[0][i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)

        loaded_model = joblib.load("elasticnet_log_p_glpk_"+str(percent)+".joblib")
        features = {loaded_model.best_estimator_.feature_names_in_[i]: loaded_model.best_estimator_.coef_[0][i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)


        proteome = X_train_solid.iloc[:, 3776:5602].loc[:, double_KO.columns[1:]]
        flux = X_train_solid.iloc[:, 5602: ]
        x_train = pd.concat([proteome, flux], axis=1)
        x_valid = X_valid_solid.iloc[:, 3776:].loc[:, x_train.columns]
        x_train.dropna(inplace=True)
        x_valid.dropna(inplace=True)

        loaded_model = joblib.load("svm_p_f_glpk_"+str(percent)+".joblib")
        result = permutation_importance(loaded_model, x_valid, y_valid_solid.loc[x_valid.index], n_repeats=10, random_state=0, n_jobs=-1)
        features = {loaded_model.best_estimator_.feature_names_in_[i]: result.importances_mean[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)

        loaded_model = joblib.load("knn_p_f_glpk_"+str(percent)+".joblib")
        result = permutation_importance(loaded_model, x_valid, y_valid_solid.loc[x_valid.index], n_repeats=10, random_state=0, n_jobs=-1)
        features = {loaded_model.best_estimator_.feature_names_in_[i]: result.importances_mean[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)
        
        loaded_model = joblib.load("lasso_log_p_f_glpk_"+str(percent)+".joblib")
        features = {loaded_model.best_estimator_.feature_names_in_[i]: loaded_model.best_estimator_.coef_[0][i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)

        loaded_model = joblib.load("elasticnet_log_p_f_glpk_"+str(percent)+".joblib")
        features = {loaded_model.best_estimator_.feature_names_in_[i]: loaded_model.best_estimator_.coef_[0][i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef = pd.concat([coef, pd.DataFrame([features])], ignore_index=True)

10


/var/folders/1g/phgxzfj511q1zyh71ggb7hxw0000gq/T/ipykernel_27149/3380375233.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y_train = y.groupby('class', group_keys=False).apply(lambda x: x.sample(n=train_size//2, random_state=42))
/var/folders/1g/phgxzfj511q1zyh71ggb7hxw0000gq/T/ipykernel_27149/3380375233.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y_valid = train_valid.groupby('class', group_

20


/var/folders/1g/phgxzfj511q1zyh71ggb7hxw0000gq/T/ipykernel_27149/3380375233.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y_train = y.groupby('class', group_keys=False).apply(lambda x: x.sample(n=train_size//2, random_state=42))
/var/folders/1g/phgxzfj511q1zyh71ggb7hxw0000gq/T/ipykernel_27149/3380375233.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y_valid = train_valid.groupby('class', group_

30


/var/folders/1g/phgxzfj511q1zyh71ggb7hxw0000gq/T/ipykernel_27149/3380375233.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y_train = y.groupby('class', group_keys=False).apply(lambda x: x.sample(n=train_size//2, random_state=42))
/var/folders/1g/phgxzfj511q1zyh71ggb7hxw0000gq/T/ipykernel_27149/3380375233.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y_valid = train_valid.groupby('class', group_

40


/var/folders/1g/phgxzfj511q1zyh71ggb7hxw0000gq/T/ipykernel_27149/3380375233.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y_train = y.groupby('class', group_keys=False).apply(lambda x: x.sample(n=train_size//2, random_state=42))
/var/folders/1g/phgxzfj511q1zyh71ggb7hxw0000gq/T/ipykernel_27149/3380375233.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y_valid = train_valid.groupby('class', group_

50


In [33]:
coef.shape

(40, 4016)

In [34]:
16*5

80

In [39]:
index = []
for p in [10, 20, 30, 40, 50]:
    for data in ['t', 't_f', 'p', 'p_f']:
        for j in ['svm', 'knn', 'lasso_log','elasticnet_log']:
            index.append(j+'_'+data+'_glpk_'+str(p))

coef.index = index
coef.to_csv("classifiers_features.csv")

In [51]:
coef_r = pd.DataFrame()
splits = [50]

for s in splits:
        
        #load and preprocess data
        [t, fluxes_t, p, fluxes_p, gr_measured_ML, y] = load_dataset()

        percent = s

        #filter based on percentile
        [gr_measured_ML, y] = filter_ko(gr_measured_ML, y, percent, 100 - percent)

        #concatenate the dataframes
        X = generate_combined_dataset(t, p, fluxes_t, fluxes_p)

        #split transcriptomics dataset into train, test and validation
        [y_train_liq, y_test_liq, y_valid_liq] = split_data(gr_measured_ML)
        X_train_liq = X.loc[y_train_liq.index, :]
        X_test_liq = X.loc[y_test_liq.index, :]
        X_valid_liq = X.loc[y_valid_liq.index, :]

        y_valid_liq = y_valid_liq['0']
        y_train_liq = y_train_liq['0']
        y_test_liq = y_test_liq['0']

        #split proteomics dataset into train, test and validation
        [y_train_solid, y_test_solid, y_valid_solid] = split_data(y)
        X_train_solid = X.loc[y_train_solid.index, :]
        X_test_solid = X.loc[y_test_solid.index, :]
        X_valid_solid = X.loc[y_valid_solid.index, :]

        y_train_solid = y_train_solid['SM']
        y_test_solid = y_test_solid['SM']
        y_valid_solid = y_valid_solid['SM']

        print(s)

        #subset train and validation for transcriptomics
        x_train = X_train_liq.iloc[:, 0:1826].loc[:, double_KO.columns[1:]]
        x_valid = X_valid_liq.iloc[:, 0:1826].loc[:, double_KO.columns[1:]]

        loaded_model = joblib.load("svm_reg_t_glpk_"+str(percent)+".joblib")
        result = permutation_importance(loaded_model, x_valid, y_valid_liq, n_repeats=10, random_state=0, n_jobs=-1)
        features = {loaded_model.best_estimator_.feature_names_in_[i]: result.importances_mean[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef_r = pd.concat([coef_r, pd.DataFrame([features])], ignore_index=True)

        loaded_model = joblib.load("lasso_reg_t_glpk_"+str(percent)+".joblib")
        features = {loaded_model.best_estimator_.feature_names_in_[i]: loaded_model.best_estimator_.coef_[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef_r = pd.concat([coef_r, pd.DataFrame([features])], ignore_index=True)


        transcriptome = X_train_liq.iloc[:, 0:1826].loc[:, double_KO.columns[1:]]
        flux = X_train_liq.iloc[:, 1826: 3776]
        x_train = pd.concat([transcriptome, flux], axis=1)
        x_valid = X_valid_liq.iloc[:, 0:3776].loc[:, x_train.columns]

        loaded_model = joblib.load("svm_reg_t_f_glpk_"+str(percent)+".joblib")
        result = permutation_importance(loaded_model, x_valid, y_valid_liq, n_repeats=10, random_state=0, n_jobs=-1)
        features = {loaded_model.best_estimator_.feature_names_in_[i]: result.importances_mean[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef_r = pd.concat([coef_r, pd.DataFrame([features])], ignore_index=True)

        loaded_model = joblib.load("lasso_reg_t_f_glpk_"+str(percent)+".joblib")
        features = {loaded_model.best_estimator_.feature_names_in_[i]: loaded_model.best_estimator_.coef_[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef_r = pd.concat([coef_r, pd.DataFrame([features])], ignore_index=True)


        x_train = X_train_solid.iloc[:, 3776:5602].loc[:, double_KO.columns[1:]]
        x_valid = X_valid_solid.iloc[:, 3776:5602].loc[:, double_KO.columns[1:]]
        x_train.dropna(inplace=True)
        x_valid.dropna(inplace=True)

        loaded_model = joblib.load("svm_reg_p_glpk_"+str(percent)+".joblib")
        result = permutation_importance(loaded_model, x_valid, y_valid_solid.loc[x_valid.index], n_repeats=10, random_state=0, n_jobs=-1)
        features = {loaded_model.best_estimator_.feature_names_in_[i]: result.importances_mean[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef_r = pd.concat([coef_r, pd.DataFrame([features])], ignore_index=True)

        loaded_model = joblib.load("lasso_reg_p_glpk_"+str(percent)+".joblib")
        features = {loaded_model.best_estimator_.feature_names_in_[i]: loaded_model.best_estimator_.coef_[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef_r = pd.concat([coef_r, pd.DataFrame([features])], ignore_index=True)


        proteome = X_train_solid.iloc[:, 3776:5602].loc[:, double_KO.columns[1:]]
        flux = X_train_solid.iloc[:, 5602: ]
        x_train = pd.concat([proteome, flux], axis=1)
        x_valid = X_valid_solid.iloc[:, 3776:].loc[:, x_train.columns]
        x_train.dropna(inplace=True)
        x_valid.dropna(inplace=True)

        loaded_model = joblib.load("svm_reg_p_f_glpk_"+str(percent)+".joblib")
        result = permutation_importance(loaded_model, x_valid, y_valid_solid.loc[x_valid.index], n_repeats=10, random_state=0, n_jobs=-1)
        features = {loaded_model.best_estimator_.feature_names_in_[i]: result.importances_mean[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef_r = pd.concat([coef_r, pd.DataFrame([features])], ignore_index=True)
        
        loaded_model = joblib.load("lasso_reg_p_f_glpk_"+str(percent)+".joblib")
        features = {loaded_model.best_estimator_.feature_names_in_[i]: loaded_model.best_estimator_.coef_[i] for i in range(len(loaded_model.best_estimator_.feature_names_in_))}
        coef_r = pd.concat([coef_r, pd.DataFrame([features])], ignore_index=True)

/var/folders/1g/phgxzfj511q1zyh71ggb7hxw0000gq/T/ipykernel_27149/3380375233.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y_train = y.groupby('class', group_keys=False).apply(lambda x: x.sample(n=train_size//2, random_state=42))
/var/folders/1g/phgxzfj511q1zyh71ggb7hxw0000gq/T/ipykernel_27149/3380375233.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y_valid = train_valid.groupby('class', group_

50


In [54]:
index = []
for p in [50]:
    for data in ['t', 't_f', 'p', 'p_f']:
        for j in ['svm', 'lasso_reg']:
            index.append(j+'_'+data+'_glpk_'+str(p))

coef_r.index = index
coef_r.to_csv("regressors_features.csv")